![opt](../resources/optimizers.jpg)

In [5]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pyplot as plt

In [6]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [7]:
import torch.optim as optim
dir(optim)


['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'Optimizer',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_multi_tensor',
 'functional',
 'lr_scheduler',
 'swa_utils']

In [8]:
def model(t_u, w, b):
    return w * t_u + b

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

In [18]:
params = torch.tensor([1.0 , 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

In [19]:
t_p = model(t_u, *params)
loss = loss_fn(t_p, t_c)
optimizer.zero_grad()
loss.backward()
optimizer.step()
params

tensor([-44.1730,  -0.8260], requires_grad=True)

In [20]:
def training_loop(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:
            params.grad.zero_()
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            params -= learning_rate * params.grad

        if epoch%500 == 0:
            print('Epoch {}, loss: {}'.format(epoch, float(loss)))
    return params


In [23]:
t_un = 0.1 * t_u
params = torch.tensor([1.0 , 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

res = training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    params=torch.tensor([1.0, 0.0], requires_grad=True),
    t_u = t_un,
    t_c = t_c
)
res

Epoch 500, loss: 7.860115051269531
Epoch 1000, loss: 3.828537940979004
Epoch 1500, loss: 3.092191219329834
Epoch 2000, loss: 2.957697868347168
Epoch 2500, loss: 2.933133840560913
Epoch 3000, loss: 2.9286484718322754
Epoch 3500, loss: 2.9278297424316406
Epoch 4000, loss: 2.9276793003082275
Epoch 4500, loss: 2.927651882171631
Epoch 5000, loss: 2.9276468753814697


tensor([  5.3671, -17.3012], requires_grad=True)